<a href="https://colab.research.google.com/github/JC1201/-Intelligent-Edge-Caching-Using-Federated-Learning-to-Predict-Content-Popularity/blob/main/TYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

#!git clone https://github.com/JC1201/-Intelligent-Edge-Caching-Using-Federated-Learning-to-Predict-Content-Popularity
# !git config --global user.name "JC1201"
# !git config --global user.email "jiehchengliew@gmail.com"
!git add .
!git commit -m "Implemented Basic model training for movie prediction"
!git push https://github.com/JC1201/-Intelligent-Edge-Caching-Using-Federated-Learning-to-Predict-Content-Popularity master

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy
from tensorflow.python.ops.gen_dataset_ops import shuffle_dataset

dataset, info = tfds.load("movielens/100k-ratings", split=["train"], with_info=True)

data_set = dataset[0]

def preprocess_data(dataset):

    user_id = tf.strings.to_number(dataset["user_id"], out_type=tf.int32)
    movie_id = tf.strings.to_number(dataset["movie_id"], out_type=tf.int32)

    #casting user rating into 0 and 1 (0 means didn't rate this movie, 1 means did rate this movie)
    did_rate = tf.cast(dataset["user_rating"] > 0, tf.int64)

    features = {
        "user_id": user_id,
        "movie_id": movie_id,
        "user_rating":  did_rate

    }
    return features

data_set = data_set.map(preprocess_data)
dataset_size = tf.data.experimental.cardinality(data_set).numpy()
print(f"Total dataset size: {dataset_size}")

num_client = 5
userS_dataset = []

for i in range(num_client):

    seed = numpy.random.seed(i)
    data_set = data_set.shuffle(buffer_size=dataset_size, seed= seed)
    userS_dataset.append(data_set)

data_set_1 =  data_set.shuffle(buffer_size=dataset_size, seed=42)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/incomplete.IOXIBM_0.1.1/movielens-train.tfrecord*..…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.
Total dataset size: 100000


In [ ]:
#size of training data and testing data
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

#extract training data and testing data
train_data_1 = data_set_1.take(int(train_size))
test_data_1 = data_set_1.skip(int(train_size))

train_data = []
test_data = []

for i in range(num_client):

    train_data.append(userS_dataset[i].take(int(train_size)))
    test_data.append(userS_dataset[i].skip(int(train_size)))


In [ ]:
userIDs = []
movieIDs = []
ratings = []

unique_user_movie_pair = set()

for example in train_data_1:

    # Convert TensorFlow tensor to a NumPy value
    user_id = example["user_id"].numpy()
    movie_id = example["movie_id"].numpy()
    rating = example["user_rating"].numpy()

    # Add movie pair only if it is uniuqe
    if (user_id, movie_id) not in unique_user_movie_pair:
        userIDs.append(user_id)
        movieIDs.append(movie_id)
        ratings.append(rating)
        unique_user_movie_pair.add((user_id, movie_id))

# Convert lists to NumPy arrays for later processing
userIDs = numpy.array(userIDs)
movieIDs = numpy.array(movieIDs)
ratings = numpy.array(ratings)

#size of user and movie in training data
num_user = max(userIDs) + 1
num_movie = max(movieIDs) + 1
num_rating = max(ratings) + 1

print(f"Number of unique users: {num_user}")
print(f"Number of unique movies: {num_movie}")


Number of unique users: 944
Number of unique movies: 1683


In [ ]:
# Get unique user and movie IDs
unique_userIDs = numpy.unique(userIDs)
unique_movieIDs = numpy.unique(movieIDs)

# Convert moive index into a seqeunce to number


# Create mapping dictionaries
user_id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_userIDs)}
movie_id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_movieIDs)}

# Map the IDs
userIDs_mapped = numpy.array([user_id_mapping[uid] for uid in userIDs])
movieIDs_mapped = numpy.array([movie_id_mapping[mid] for mid in movieIDs])

# New dimensions
num_user = len(unique_userIDs)
num_movie = len(unique_movieIDs)

print(f"Number of users: {num_user}, Number of movies: {num_movie}")


Number of users: 943, Number of movies: 1653


In [ ]:
def integrate_feature_into_matrix(userIDs, movieIDs, ratings, num_user, num_movie):

    # Create a 2D matrix filled with zeros
    user_movie_matrix = numpy.zeros((num_user, num_movie), dtype=numpy.int32)

    # Populate the matrix
    for userID, movieID, rating in zip(userIDs, movieIDs, ratings):
        user_movie_matrix[int(userID), int(movieID)] = rating

    return user_movie_matrix

data_set_matrix = integrate_feature_into_matrix(userIDs_mapped, movieIDs_mapped, ratings, num_user, num_movie)
train_matrix = data_set_matrix.reshape((1, num_user, num_movie, 1))
print(train_matrix.shape)

# train_percentage = 0.8
# train_size = int(num_user * train_percentage)  # Number of users for training
# test_size = num_user - train_size

# print(f"Number of users for training: {train_size}")
# print(f"Number of users for testing: {test_size}")

# train_matrix = data_set_matrix[:, :train_size, :, :]
# test_matrix = data_set_matrix[:, test_size:, :, :]

# # train_matrix = integrate_feature_into_matrix(userIDs_mapped, movieIDs_mapped, ratings, num_user, num_movie)
# # test_matrix = test_matrix.reshape((1, num_user, num_movie, 1))
# print(train_matrix.shape)
# print(test_matrix.shape)


(1, 943, 1653, 1)


In [ ]:
def build_cnn_model(num_user, num_movie):

    input_matrix = tf.keras.layers.Input(shape=(num_user, num_movie, 1),batch_size = 32, name="user_id")

    cnn_layer = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_matrix)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(4, 4))(cnn_layer) # Added MaxPooling

    cnn_layer = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(4, 4))(cnn_layer) # Added MaxPooling

    cnn_layer = tf.keras.layers.Flatten()(cnn_layer)

    dense_layer = tf.keras.layers.Dense(128, activation='relu')(cnn_layer)

    output = tf.keras.layers.Dense(num_movie, activation='sigmoid', name="movie_scores")(dense_layer)
    model = tf.keras.models.Model(inputs=input_matrix, outputs=output, name="MoviePopularityModel")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error',  # For regression tasks
        metrics=['accuracy']  # Mean Absolute Error for monitoring
    )

    return model


model = build_cnn_model(num_user, num_movie)

model.fit(
    train_matrix,
    train_matrix,  # For self-supervised learning
    epochs=10,
    batch_size=32
)


Epoch 1/10


In [ ]:
# Predict movie scores
predictions = model.predict(train_matrix)

# Aggregate scores across users
movie_scores = predictions.mean(axis=0)

# Get top 50 movie indices
# Top 50 in descending ordere
movie_score = numpy.argsort(movie_scores)[:][::-1]

print("Predicted Movies Ranking(by index):")
for rank, movie_idx in enumerate(movie_score, 1):
    print(f"{rank}: Movie Index {movie_idx}, Score {movie_scores[movie_idx] } ")




In [ ]:
# loss, mae = model.evaluate(train_matrix, test_matrix)  # For self-supervised learning, use test_matrix as both inputs and targets
# print(f"Test Loss: {loss}")
# print(f"Test MAE: {mae}")

# predicted_scores = model.predict(test_matrix)
# print(f"Score: {predicted_scores}")


What to do next

IF the cach hit rate is high enough

try with multiple user